In [1]:
# made by Gibeom LEE, HI LAB
# csv파일 여러개를 일정한 간격을 두어 합쳐줍니다. (다른 열)

In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
#폴더, 파일명, 간격 범위 기입
folder = r"C:\Users\GCU\OneDrive - gachon.ac.kr\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\3. Material selection\250729_UC1모양으로 코팅별\10k_csv\Krytox103"

file_name_str = "OP103_6_40 103_6cm_8mLmin 103_6cm_12mLmin" #"" 안에 하나의 csv파일

gap_value = 0.2 #0.2 = 20 % 만큼 행 띄우기

In [4]:
output_path = os.path.join(folder, "00Merge.csv")

def get_robust_time_step(df):
    """총 시간과 행 수로부터 평균 time_step 계산"""
    time_col = df.iloc[:, 0].values
    total_time = round(time_col[-1])           # 총 시간 = 마지막 값 반올림
    n_rows = len(time_col)                     # 전체 샘플 수
    time_step = total_time / n_rows
    return time_step, n_rows, total_time

def merge_data(file_name_str, folder, output_path):
    file_names = file_name_str.split()
    file_paths = [os.path.join(folder, f"{name}.csv") for name in file_names]
    dataframes = [pd.read_csv(fp, header=None) for fp in file_paths]

    # 첫 번째 파일 기준으로 time_step, 샘플 수 계산
    time_step, n_rows, total_time = get_robust_time_step(dataframes[0])
    gap = int(gap_value * n_rows)

    total_rows = n_rows + (len(dataframes) - 1) * (n_rows + gap)
    n_cols = len(dataframes) + 1
    result_array = np.full((total_rows, n_cols), np.nan)

    # 시간 축 생성: 0초부터 시작, time_step 간격으로 증가
    result_array[:, 0] = np.arange(0, total_rows) * time_step

    # 데이터 병합
    row_pos = 0
    for i, df in enumerate(dataframes):
        y_values = df.iloc[:, 1].values  # 전압 값 (2번째 열 기준)
        result_array[row_pos:row_pos + n_rows, i + 1] = y_values
        row_pos += n_rows + gap

    # 저장
    pd.DataFrame(result_array).to_csv(output_path, index=False, header=False, float_format="%.6f")
    print(f"✅ 병합된 파일이 저장되었습니다: {output_path}")

# 실행
merge_data(file_name_str, folder, output_path)


✅ 병합된 파일이 저장되었습니다: C:\Users\GCU\OneDrive - gachon.ac.kr\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\3. Material selection\250729_UC1모양으로 코팅별\10k_csv\Krytox103\00Merge.csv
